In [1]:
# load all packages in Julia
import Pkg; Pkg.add("VegaLite"); Pkg.add("PrettyTables")
using JuMP, HiGHS
using Plots;
using VegaLite  # to make some nice plots
using DataFrames, CSV, PrettyTables
ENV["COLUMNS"]=120; # Set so all columns of DataFrames and Matrices are displayed

   Resolving package versions...
  No Changes to `C:\Users\44780\.julia\environments\v1.9\Project.toml`
  No Changes to `C:\Users\44780\.julia\environments\v1.9\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\44780\.julia\environments\v1.9\Project.toml`
  No Changes to `C:\Users\44780\.julia\environments\v1.9\Manifest.toml`


In [35]:
#open csv files to define sets, lines, gens and and loads/demand
datadir = joinpath("ReducedNetwork") 
gens = CSV.read(joinpath(datadir,"GenRedNet2.csv"), DataFrame);
lines = CSV.read(joinpath(datadir,"TransRedNet.csv"), DataFrame);
loads = CSV.read(joinpath(datadir,"demandRedNet.csv"), DataFrame);


# Rename all columns to lowercase (by convention)
for f in [gens, lines, loads]
    rename!(f,lowercase.(names(f)))
end

# create generator ids #changed to read columns rather than rows
gens.id = 1:nrow(gens);

# create line ids 
lines.id = 1:nrow(lines);
# add set of rows for reverse direction with same parameters

lines2 = copy(lines)
lines2.f = lines2.fromnode
lines2.fromnode = lines.tonode
lines2.tonode = lines.fromnode
lines2 = lines2[:,names(lines)]
append!(lines,lines2)

#lines.capacity = 0.4*lines.capacity
# calculate simple susceptance, ignoring resistance as earlier 
lines.b = 1 ./ lines.reactance

# keep only a single time period
#loads = loads[:,["connnode","interval-1_load"]]
#rename!(loads,"interval-1_load" => "demand");

lines

Row,fromnode,tonode,resistance,reactance,contingencymarked,capacity,id,b
,Int64,Int64,Float64,Float64,Int64,Int64,Int64,Float64
1,1,2,0.0122,0.02,1,10000,1,50.0
2,1,3,0.007,0.15,1,10000,2,6.66667
3,1,2,0.0122,0.02,1,10000,3,50.0
4,1,3,0.007,0.15,1,10000,4,6.66667
5,2,4,0.0004,0.065,1,10000,5,15.3846
6,2,4,0.0004,0.065,1,10000,6,15.3846
7,4,7,0.00211,0.0135,1,10000,7,74.0741
8,4,6,0.0013,0.023,1,10000,8,43.4783
9,4,6,0.0013,0.023,1,10000,9,43.4783


In [3]:
gens

Row,connnode,c2,c1,c0,pgmax,pgmin,rgmax,rgmin,pgprev,id
,Int64,Int64,Float64,Int64,Float64,Int64,Int64,Int64,Int64,Int64
1,1,0,12.0,0,668.0,0,100,-100,0,1
2,1,0,395.37,0,554.28,0,100,-100,0,2
3,1,0,86.38,0,300.0,0,100,-100,0,3
4,1,0,32.0,0,18.72,0,100,-100,0,4
5,2,0,37.0,0,50.6,0,100,-100,0,5
6,2,0,400.68,0,1524.0,0,100,-100,0,6
7,2,0,400.68,0,12.0,0,100,-100,0,7
8,3,0,32.0,0,550.99,0,100,-100,0,8
9,3,0,27.0,0,19.0,0,100,-100,0,9


In [13]:
loads

Row,connnode,demand
,Int64,Float64
1,1,-2036.63
2,2,-1734.71
3,3,-877.9
4,4,-1334.16
5,5,-1701.55
6,6,-1244.77
7,7,-1341.61
8,8,-3669.5
9,9,-1960.02


In [4]:
#define optimization function 
#Nodal pricing - thiswill form the second step of the National pricing (re-dispatch)
#Function to solve DC F problem using IEEE test cases
#Inputs:
    #gen_info -- dataframe with generator info
    #line_info -- dataframe with transmission lines info
    #loads  -- dataframe with load info
#
function np_attempt(gens, lines, loads)
    NP = Model(HiGHS.Optimizer) # You could use Clp as well, with Clp.Optimizer
    
    # Define sets based on data
      # Set of generator buses
    G = gens.id #the way it defines gens are by looking at which bus it's connected to - list gens as unique elements then create another matrix showing which gens are associated with which node in the system
                #set to id instead of connnode
    
    
      # Set of all nodes
    N = sort(union((lines.fromnode), 
            (lines.tonode))) #took out unique
    
      # sets J_i and G_i will be described using dataframe indexing below

    # Define per unit base units for the system 
    # used to convert from per unit values to standard unit
    # values (e.g. p.u. power flows to MW/MVA)
    baseMVA = 100 # base MVA is 100 MVA for this system *LOOK INTO THIS FURTHER*
    
    # Decision variables   
    @variables(NP, begin
        GEN[G]  >= 0     # generation        
        # Note: we assume Pmin = 0 for all resources for simplicty here
        THETA[N]         # voltage phase angle of bus
        FLOW[N,N]        # flows along each line
    end)
    
    # Create slack bus with reference angle = 0; use bus 1 with generator
    fix(THETA[1],0)
                
    # Objective function
    @objective(NP, Min, 
        sum(gens[g,:c1] * GEN[g] for g in G)
    )
    
    # Supply demand balances
    @constraint(NP, cBalance[i in N], 
        sum(GEN[g] for g in gens[gens.connnode .== i,:id]) 
            + sum(load for load in loads[loads.connnode .== i,:demand]) 
        == sum(FLOW[i,j] for j in lines[lines.fromnode .== i,:tonode])
    )

    # Max generation constraint
    @constraint(NP, cMaxGen[g in G],
                   GEN[g] <= gens[g,:pgmax])
    
 
    


    # Solve statement (! indicates runs in place)
    optimize!(NP)

    # Output variables
    generation = DataFrame(
        node = gens.connnode,
        gen = value.(GEN).data[gens.connnode]
        )
    
    angles = value.(THETA).data
    
    flows = DataFrame(
        fbus = lines.fromnode,
        tbus = lines.tonode,
        flow = baseMVA * lines.b .* (angles[lines.fromnode] .- 
                        angles[lines.tonode]))

    #load payment
    #loadpay = sum(dual.(cBalance)*(loads for loads in loads[loads.connnode, :demand]))
    

    #congestion cost
    
    
    #Total Gen prof
    #genprof = sum(GEN[g] for g in gens[gens.connnode .== i,:id]) * sum()
    
    # We output the marginal values of the demand constraints, 
    # which will in fact be the prices to deliver power at a given bus.
    prices = DataFrame(
        node = N,
        value = dual.(cBalance).data)

    # Return the solution and objective as named tuple
    return (
        generation = generation, 
        angles,
        flows,
        prices,
        #loadpay,
        cost = objective_value(NP),
        status = termination_status(NP)
    )
end

np_attempt (generic function with 1 method)

In [5]:
#solve 
sol = np_attempt(gens, lines, loads);

Running HiGHS 1.5.3 [date: 1970-01-01, git hash: 45a127b78]
Copyright (c) 2023 HiGHS under MIT licence terms
Presolving model
0 rows, 66 cols, 0 nonzeros
0 rows, 0 cols, 0 nonzeros
Presolve : Reductions: rows 0(-95); columns 0(-936); elements 0(-232) - Reduced to empty
Solving the original LP from the solution after postsolve
Model   status      : Optimal
Objective value     :  0.0000000000e+00
HiGHS run time      :          0.00


In [6]:
#generation output
sol.generation

Row,node,gen
,Int64,Float64
1,1,0.0
2,1,0.0
3,1,0.0
4,1,0.0
5,2,0.0
6,2,0.0
7,2,0.0
8,3,0.0
9,3,0.0


In [7]:
#prices output
sol.prices

Row,node,value
,Int64,Float64
1,1,-0.0
2,2,-0.0
3,3,-0.0
4,4,-0.0
5,5,-0.0
6,6,-0.0
7,7,-0.0
8,8,-0.0
9,9,-0.0


In [8]:
#flows output
sol.flows

Row,fbus,tbus,flow
,Int64,Int64,Float64
1,1,2,0.0
2,1,3,0.0
3,1,2,0.0
4,1,3,0.0
5,2,4,0.0
6,2,4,0.0
7,4,7,0.0
8,4,6,0.0
9,4,6,0.0


In [9]:
#sum of gens
sum(sol.generation[:,2])

0.0

In [10]:
#sum of prices per node
sum(sol.prices[:,2])

0.0

In [11]:
#sum of flows per node
sum(sol.flows[:,3])

0.0

In [32]:
#test using old 36 bus network 
datadir = joinpath("testcase") 
gens = CSV.read(joinpath(datadir,"Gen36.csv"), DataFrame);
lines = CSV.read(joinpath(datadir,"Tran36_b_csv.csv"), DataFrame);
loads = CSV.read(joinpath(datadir,"Load36_csv.csv"), DataFrame);


# Rename all columns to lowercase (by convention)
for f in [gens, lines, loads]
    rename!(f,lowercase.(names(f)))
end

# create generator ids 
gens.id = 1:nrow(gens);

# create line ids 
lines.id = 1:nrow(lines);
# add set of rows for reverse direction with same parameters

lines2 = copy(lines)
lines2.f = lines2.fromnode
lines2.fromnode = lines.tonode
lines2.tonode = lines2.f
lines2 = lines2[:,names(lines)]
append!(lines,lines2)

# calculate simple susceptance, ignoring resistance as earlier 
#lines.b = 1 ./ lines.reactance

# keep only a single time period
loads = loads[:,["connnode","interval-1_load"]]
rename!(loads,"interval-1_load" => "demand");

lines

Row,fromnode,tonode,resistance,reactance,contingencymarked,capacity,id,b
,Int64,Int64,Float64,Float64,Int64,Int64,Int64,Float64
1,1,2,0.00266925,0.0323779,1,10000,1,0.0196113
2,1,5,0.00287887,0.0286089,1,10000,2,0.0103084
3,2,3,0.00131306,0.0190229,1,10000,3,0.0102966
4,2,6,0.000803188,0.0149712,1,10000,4,0.00476255
5,3,10,0.000399063,0.00548875,1,10000,5,0.00238071
6,4,5,0.00114981,0.0183772,1,10000,6,0.0126698
7,4,6,0.00209787,0.0318498,1,10000,7,0.0106544
8,4,14,0.00220469,0.0290109,1,10000,8,0.00897751
9,5,6,0.000706313,0.00961025,1,10000,9,0.00913257


In [26]:
#attempt 2 using PyPSA-GB network - works properly for the 36 bus network
#=
Function to solve DC OPF problem using IEEE test cases
Inputs:
    gen_info -- dataframe with generator info
    line_info -- dataframe with transmission lines info
    loads  -- dataframe with load info
=#
function dcopf_ieee(gens, lines, loads)
    DCOPF = Model(HiGHS.Optimizer) # You could use Clp as well, with Clp.Optimizer
    
    # Define sets based on data
      # Set of generator buses
    G = gens.id #the way it defines gens are by looking at which bus it's connected to - list gens as unique elements then create another matrix showing which gens are associated with which node in the system
                #set to id instead of connnode
    
      # Set of all nodes
    N = sort(union(unique(lines.fromnode), 
            unique(lines.tonode))) 
    
      # sets J_i and G_i will be described using dataframe indexing below

    # Define per unit base units for the system 
    # used to convert from per unit values to standard unit
    # values (e.g. p.u. power flows to MW/MVA)
    baseMVA = 100 # base MVA is 100 MVA for this system 
    
    # Decision variables   
    @variables(DCOPF, begin
        GEN[G]  >= 0     # generation        
        # Note: we assume Pmin = 0 for all resources for simplicty here
        THETA[N]         # voltage phase angle of bus
        FLOW[N,N]        # flows between all pairs of nodes
    end)
    
    # Create slack bus with reference angle = 0; use bus 1 with generator
    fix(THETA[1],0)
                
    # Objective function
    @objective(DCOPF, Min, 
        sum( gens[g,:c1] * GEN[g] for g in G)
    )
    
    # Supply demand balances
    @constraint(DCOPF, cBalance[i in N], 
        sum(GEN[g] for g in gens[gens.connnode .== i,:id]) 
            + sum(load for load in loads[loads.connnode .== i,:demand]) 
        == sum(FLOW[i,j] for j in lines[lines.fromnode .== i,:tonode])
    )

    # Max generation constraint
    @constraint(DCOPF, cMaxGen[g in G],
                    GEN[g] <= gens[g,:pgmax])
    
    # Flow constraints on each branch 
    @constraint(DCOPF, cLineFlows[l in 1:nrow(lines)],
            FLOW[lines[l,:fromnode],lines[l,:tonode]] == 
            baseMVA * lines[l,:b] * 
            (THETA[lines[l,:fromnode]] - THETA[lines[l,:tonode]])
    )
    
    # Max line flow constraints
    @constraint(DCOPF, cLineLimits[l in 1:nrow(lines)], 
            FLOW[lines[l,:fromnode],lines[l,:tonode]] <=
            lines[l,:capacity]
    ) 


    # Solve statement (! indicates runs in place)
    optimize!(DCOPF)

    # Output variables
    generation = DataFrame(
        node = gens.connnode,
        gen = value.(GEN).data[gens.connnode]
        )
    
    angles = value.(THETA).data
    
    flows = DataFrame(
        fbus = lines.fromnode,
        tbus = lines.tonode,
        flow = baseMVA * lines.b .* (angles[lines.fromnode] .- 
                        angles[lines.tonode]))
    
    # We output the marginal values of the demand constraints, 
    # which will in fact be the prices to deliver power at a given bus.
    prices = DataFrame(
        node = N,
        value = dual.(cBalance).data)

    # Return the solution and objective as named tuple
    return (
        generation = generation, 
        angles,
        flows,
        prices,
        cost = objective_value(DCOPF),
        status = termination_status(DCOPF)
    )
end

dcopf_ieee (generic function with 1 method)

In [36]:
solution = dcopf_ieee(gens, lines, loads);

Running HiGHS 1.5.3 [date: 1970-01-01, git hash: 45a127b78]
Copyright (c) 2023 HiGHS under MIT licence terms
Presolving model
226 rows, 193 cols, 750 nonzeros
140 rows, 181 cols, 497 nonzeros
127 rows, 175 cols, 454 nonzeros
Presolve : Reductions: rows 127(-364); columns 175(-761); elements 454(-570)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0    -7.2222137846e+00 Pr: 127(3.71204e+06); Du: 0(1.52771e-11) 0s
        132     5.7705897923e+07 0s
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
        132     5.7706271900e+07 Pr: 31(2.09872e+06) 0s
        136     1.3258425953e+08 0s
Model   status      : Unknown
Simplex   iterations: 136
Objective value     :  1.3258364344e+08
HiGHS run time      :          0.00


In [37]:
solution.generation #pypsa-gb results below

Row,node,gen
,Int64,Float64
1,1,554.28
2,1,554.28
3,1,554.28
4,1,554.28
5,2,300.0
6,2,300.0
7,2,300.0
8,3,18.72
9,3,18.72


In [38]:
solution.prices

Row,node,value
,Int64,Float64
1,1,400.68
2,2,400.68
3,3,400.68
4,4,400.68
5,5,-0.0
6,6,635.862
7,7,-175.514
8,8,998.425
9,9,400.68


In [31]:
solution.flows

Row,fbus,tbus,flow
,Int64,Int64,Float64
1,1,2,3.34392e6
2,1,3,4.42387e5
3,1,2,3.34392e6
4,1,3,4.42387e5
5,2,4,-9551.22
6,2,4,-9551.22
7,4,7,13008.0
8,4,6,5085.15
9,4,6,5085.15
